In [25]:
import json
import numpy as np
import xml.etree.ElementTree as ET

In [50]:
def ExtractSumoNet(tree_path):

    # 读取XML文件
    tree = ET.parse(tree_path)
    root = tree.getroot()

    # 遍历XML元素
    roads={}
    for element in root:
        if element.tag=='edge':
            for attr,value in element.attrib.items():
                if attr=='id' and len(value.split('_'))==3 and 'road' in value:
                    for at,va in element[0].attrib.items():
                        if at=='shape':
                            p1=va.split(' ')[0].split(',')
                    for at,va in element[-1].attrib.items():
                        if at=='shape':
                            p2=va.split(' ')[1].split(',')
                    roads[value]=[float(i) for i in p1+p2]


    return roads

In [109]:
tree_path='./SH2_edit.net.xml'


roads=ExtractSumoNet(tree_path)
roadnet=json.load(open('./SH2/roadnet.json','r'))

# 修改道路
cityroads=roadnet['roads']
for road in cityroads:
    points=roads[road['id']]
    road['points']=[{'x':points[0],'y':points[1]},{'x':points[2],'y':points[3]}]


# 修改交叉口
intersections=roadnet['intersections']
tlsPos={}
tls=dict(zip([k for k in roads.keys()],[v[2:] for v in roads.values()]))
inters=[k.split('_')[-1] for k in tls.keys()]
for inter in inters:
    center=np.array([tls[k] for k in tls.keys() if inter==k.split('_')[-1]])
    x,y=center[:,0].mean(),center[:,1].mean()
    tlsPos.update({'inter_'+inter:{"x":x,"y":y}})  

for it in intersections:
    it['point']=tlsPos[it['id']]

roadnet['roads']=cityroads
roadnet['intersections']=intersections
roadnet=json.dumps(roadnet,indent=2)
with open('./SH2/roadnet.json','w') as f:
    f.write(roadnet)
